# Задание
Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

In [13]:
import hyperopt as ho
import numpy as np
import optuna as ot
import pandas as pd
from sklearn import ensemble, linear_model, metrics, model_selection

In [14]:
data_df = pd.read_csv('../data/train_sem09.csv')

display(data_df.describe())
data_df.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
count,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,...,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000
mean,0.542255,0.076948,0.592436,0.068142,0.038990,0.212112,0.686653,0.274713,0.455133,0.749517,...,0.026926,0.014663,0.013863,0.021861,0.015196,0.016796,0.012263,0.011730,0.020261,0.011197
std,0.498278,0.079989,0.105860,0.078414,0.115885,0.102592,0.078702,0.090017,0.162731,0.071702,...,0.161889,0.120215,0.116938,0.146249,0.122348,0.128522,0.110074,0.107683,0.140911,0.105236
min,0.000000,0.000000,0.282128,0.000000,0.000000,0.002630,0.137873,0.006130,0.000000,0.275590,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.033300,0.517811,0.000000,0.000000,0.138118,0.625627,0.207374,0.378062,0.707339,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.066700,0.585989,0.050000,0.000000,0.190926,0.674037,0.277845,0.499942,0.738961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.100000,0.668395,0.100000,0.000000,0.261726,0.740663,0.335816,0.569962,0.788177,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,0.964381,0.950000,1.000000,1.000000,0.994735,0.790831,0.989870,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [41]:
random_state = 42
max_iter = 50

X = data_df.drop(columns='Activity')
y = data_df['Activity']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=random_state, stratify=y, test_size=0.2)

# Метод GridSeachCV

In [16]:
#Объявляем фунцию, которая будет возвращать модель данных с подобранными параметрами (максимум метрики f1)
def grid_search(estimator, param_grid, X, y):
    params_searcher = model_selection.GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    cv=5,
    scoring='f1'
    )
    params_searcher.fit(X, y)
    
    return params_searcher.best_estimator_

## Подбор гиперпараметров для логистической регрессии

In [17]:
%%time

#Создаем модель данных
model_lr01 = linear_model.LogisticRegression(random_state=random_state, max_iter=max_iter)

#Задаем сетку параметров
param_grid_lg = [
    {'penalty': ['l2', 'none'],
    'solver': ['lbfgs', 'sag'],
    'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

#Получаем настроенную и обученную модель
model_lg01 = grid_search(model_lr01, param_grid_lg, X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

CPU times: user 3min 4s, sys: 9.1 s, total: 3min 13s
Wall time: 3min 37s


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [18]:
#Расчитываем метрики модели на тренировочной и тестовой выборках
print(f'Метрика f1 для LG на выборке train: {metrics.f1_score(y_train, model_lg01.predict(X_train))}')
print(f'Метрика f1 для LG на выборке test: {metrics.f1_score(y_test, model_lg01.predict(X_test))}')

Метрика f1 для LG на выборке train: 0.8228264117119809
Метрика f1 для LG на выборке test: 0.780952380952381


## Подбор гиперпараметров для случайного леса

In [19]:
%%time

#Создаем модель
model_rfc01 = ensemble.RandomForestClassifier(random_state=random_state)

#Задаем сетку параметров
param_grid_rfc = [
    {'n_estimators': list(range(80, 200, 30)),
    'min_samples_leaf': [5, 7],
    'max_depth': list(np.linspace(10, 30, 5, dtype=int))}
]

#Получаем настроенную и обученную модель
model_rfc01 = grid_search(model_rfc01, param_grid_rfc, X_train, y_train)

CPU times: user 3min 55s, sys: 7.52 s, total: 4min 2s
Wall time: 4min 57s


In [20]:
#Расчитываем метрики модели на тренировочной и тестовой выборках
print(f'Метрика f1 для RFC на выборке train: {metrics.f1_score(y_train, model_rfc01.predict(X_train))}')
print(f'Метрика f1 для RFC на выборке test: {metrics.f1_score(y_test, model_rfc01.predict(X_test))}')

Метрика f1 для RFC на выборке train: 0.9382039573820395
Метрика f1 для RFC на выборке test: 0.8004866180048661


# Метод RandomizedSearchCV

In [21]:
max_iter = 30

#Объявляем фунцию, которая будет возвращать модель данных с подобранными параметрами (максимум метрики f1)
def randomized_search(estimator, param_distributions, X, y):
    params_searcher = model_selection.RandomizedSearchCV(
    estimator=estimator,
    param_distributions=param_distributions,
    cv=5,
    scoring='f1',
    n_iter=max_iter
    )
    params_searcher.fit(X, y)
    
    return params_searcher.best_estimator_

## Подбор гиперпараметров для логистической регрессии

In [22]:
%%time

#Создаем модель данных
model_lr02 = linear_model.LogisticRegression(random_state=random_state, max_iter=max_iter)

#Задаем сетку параметров
param_distributions_lg = [
    {'penalty': ['l2', 'none'],
    'solver': ['lbfgs', 'sag'],
    'C': list(np.linspace(0.01, 1, 10, dtype=float))}
]

#Получаем настроенную и обученную модель
model_lg02 = randomized_search(model_lr02, param_distributions_lg, X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was

CPU times: user 1min 54s, sys: 7.1 s, total: 2min 2s
Wall time: 2min 9s


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [23]:
#Расчитываем метрики модели на тренировочной и тестовой выборках
print(f'Метрика f1 для LG на выборке train: {metrics.f1_score(y_train, model_lg02.predict(X_train))}')
print(f'Метрика f1 для LG на выборке test: {metrics.f1_score(y_test, model_lg02.predict(X_test))}')

Метрика f1 для LG на выборке train: 0.8230378991345867
Метрика f1 для LG на выборке test: 0.7800237812128419


## Подбор гиперпараметров для случайного леса

In [24]:
%%time

#Создаем модель
model_rfc02 = ensemble.RandomForestClassifier(random_state=random_state)

#Задаем сетку параметров
param_distributions_rfc = [
    {'n_estimators': list(range(80, 200, 30)),
    'min_samples_leaf': [5, 7],
    'max_depth': list(np.linspace(10, 30, 5, dtype=int))}
]

#Получаем настроенную и обученную модель
model_rfc02 = randomized_search(model_rfc02, param_distributions_rfc, X_train, y_train)

CPU times: user 3min 3s, sys: 5.83 s, total: 3min 9s
Wall time: 3min 52s


In [25]:
#Расчитываем метрики модели на тренировочной и тестовой выборках
print(f'Метрика f1 для RFC на выборке train: {metrics.f1_score(y_train, model_rfc02.predict(X_train))}')
print(f'Метрика f1 для RFC на выборке test: {metrics.f1_score(y_test, model_rfc02.predict(X_test))}')

Метрика f1 для RFC на выборке train: 0.9382039573820395
Метрика f1 для RFC на выборке test: 0.8004866180048661


# Метод Hyperopt

## Подбор гиперпараметров для логистической регрессии

In [26]:
#Функция оценки качества модели
def lr_score(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    model_lr = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=max_iter)
    model_lr.fit(X, y)

    return -metrics.f1_score(y, model_lr.predict(X))

In [27]:
%%time

#Задаем пространство признаков. Делаем так, что бы оно охватывало все признака из предыдущих экспериметов
space_lr = {
    'penalty': ho.hp.choice('penalty', ['none', 'l2']),
    'solver' : ho.hp.choice('solver', ['sag', 'lbfgs']),
    'C': ho.hp.uniform('C', 0.01, 1)  
}

#Подбираем оптимальные параметры
trials_lr = ho.Trials()
best_params_lr = ho.fmin(
    lr_score,
    space=space_lr,
    max_evals=max_iter,
    trials=trials_lr,
    rstate=np.random.default_rng(random_state)
    )

best_params_lr = ho.space_eval(space_lr, best_params_lr)

TPE is being used as the default algorithm.


  3%|▎         | 1/30 [00:02<01:05,  2.26s/trial, best loss: -0.8700393820054528]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(



  7%|▋         | 2/30 [00:02<00:30,  1.07s/trial, best loss: -0.8700393820054528]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(



...

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(



 97%|█████████▋| 29/30 [00:40<00:01,  1.80s/trial, best loss: -0.8739750986941998]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(



100%|██████████| 30/30 [00:41<00:00,  1.40s/trial, best loss: -0.8739750986941998]
CPU times: user 35.5 s, sys: 2 s, total: 37.5 s
Wall time: 42.3 s


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



In [28]:
#Создаем модель с оптимальными параметрами
model_lr03 = linear_model.LogisticRegression(
    penalty=best_params_lr['penalty'],
    solver=best_params_lr['solver'],
    C=best_params_lr['C'],
    random_state=random_state,
    max_iter=max_iter
)

#Обучаем модель
model_lr03.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=0.08292441519601887, max_iter=30, penalty='none',
                   random_state=42, solver='sag')

In [29]:
#Расчитываем метрики модели на тренировочной и тестовой выборках
print(f'Метрика f1 для LR на выборке train: {metrics.f1_score(y_train, model_lr03.predict(X_train))}')
print(f'Метрика f1 для LR на выборке test: {metrics.f1_score(y_test, model_lr03.predict(X_test))}')

Метрика f1 для LR на выборке train: 0.8739750986941998
Метрика f1 для LR на выборке test: 0.7752126366950183


## Подбор гиперпараметров для случайного леса

In [30]:
#Функция оценки качества модели
def rfc_score(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    params = {
        'n_estimators': int(params['n_estimators']),
        'min_samples_leaf': int(params['min_samples_leaf']),
        'max_depth': int(params['max_depth'])
        }

    model_rfc = ensemble.RandomForestClassifier(**params, random_state=random_state)
    model_rfc.fit(X, y)

    return -metrics.f1_score(y, model_rfc.predict(X))

In [31]:
%%time

#Задаем пространство признаков. Делаем так, что бы оно охватывало все признака из предыдущих экспериметов
space_rfc = {
    'n_estimators': ho.hp.quniform('n_estimators', 80, 250, 1),
    'max_depth' : ho.hp.quniform('max_depth', 10, 40, 1),
    'min_samples_leaf': ho.hp.quniform('min_samples_leaf', 3, 10, 1)  
}

#Подбираем оптимальные параметры
trials_rfc = ho.Trials()
best_params_rfc = ho.fmin(
    rfc_score,
    space=space_rfc,
    max_evals=max_iter,
    trials=trials_rfc,
    rstate=np.random.default_rng(random_state)
    )

TPE is being used as the default algorithm.


100%|██████████| 30/30 [01:07<00:00,  2.25s/trial, best loss: -0.975594874923734] 
CPU times: user 58.5 s, sys: 1.58 s, total: 1min
Wall time: 1min 7s


In [32]:
#Создаем модель с оптимальными параметрами
model_rfc03 = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best_params_rfc['n_estimators']),
    max_depth=int(best_params_rfc['max_depth']),
    min_samples_leaf=int(best_params_rfc['min_samples_leaf'])
)

#Обучаем модель
model_rfc03.fit(X_train, y_train)

RandomForestClassifier(max_depth=25, min_samples_leaf=3, n_estimators=183,
                       random_state=42)

In [33]:
#Расчитываем метрики модели на тренировочной и тестовой выборках
print(f'Метрика f1 для RFC на выборке train: {metrics.f1_score(y_train, model_rfc03.predict(X_train))}')
print(f'Метрика f1 для RFC на выборке test: {metrics.f1_score(y_test, model_rfc03.predict(X_test))}')

Метрика f1 для RFC на выборке train: 0.975594874923734
Метрика f1 для RFC на выборке test: 0.7980535279805353


# Метод OPTUNA

## Подбор гиперпараметров для логистической регрессии

In [34]:
#Функция оценки качества модели

def score_lr_optuna(trial):
      space = {
        'penalty': trial.suggest_categorical('penalty', ['none', 'l2']),
        'solver': trial.suggest_categorical('solver', ['sag', 'lbfgs']),
        'C': trial.suggest_float('C', 0.01, 1)
        }

      model = linear_model.LogisticRegression(**space, random_state=random_state, max_iter=max_iter)
      model.fit(X_train, y_train)
      
      #Применяем кросс-валидацию при оценке качества модели
      score = model_selection.cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()
      
      return score

In [35]:
%%time

#Подбираем оптимальные параметры
study01 = ot.create_study(study_name="LogisticRegression", direction="maximize")
study01.optimize(score_lr_optuna, n_trials=max_iter)

[I 2023-02-08 02:03:07,903] A new study created in memory with name: LogisticRegression
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='no

CPU times: user 3min 17s, sys: 9.09 s, total: 3min 26s
Wall time: 3min 37s


In [36]:
#Создаем модель с оптимальными параметрами
model_lr04 = linear_model.LogisticRegression(**study01.best_params, random_state=random_state, max_iter=max_iter)

#Обучаем модель
model_lr04.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=0.020193496871209193, max_iter=30, random_state=42,
                   solver='sag')

In [37]:
#Расчитываем метрики модели на тренировочной и тестовой выборках
print(f'Метрика f1 для LR на выборке train: {metrics.f1_score(y_train, model_lr04.predict(X_train))}')
print(f'Метрика f1 для LR на выборке test: {metrics.f1_score(y_test, model_lr04.predict(X_test))}')

Метрика f1 для LR на выборке train: 0.8318849265807612
Метрика f1 для LR на выборке test: 0.7805456702253855


## Подбор гиперпараметров для случайного леса

In [43]:
#Функция оценки качества модели

def score_rfc_optuna(trial):
      space = {
        'n_estimators': trial.suggest_int('n_estimators', 80, 250, 1),
        'max_depth': trial.suggest_int('max_depth', 10, 40, 1),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 10, 1)
        }

      model = ensemble.RandomForestClassifier(**space, random_state=random_state)
      model.fit(X_train, y_train)
      
      #Применяем кросс-валидацию при оценке качества модели
      score = model_selection.cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()
      
      return score

In [44]:
%%time

#Подбираем оптимальные параметры
study02 = ot.create_study(study_name="RandomForestClassifier", direction="maximize")
study02.optimize(score_rfc_optuna, n_trials=max_iter)

[I 2023-02-08 02:16:55,246] A new study created in memory with name: RandomForestClassifier
[I 2023-02-08 02:17:09,453] Trial 0 finished with value: 0.8155294831202671 and parameters: {'n_estimators': 147, 'max_depth': 15, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8155294831202671.
[I 2023-02-08 02:17:27,153] Trial 1 finished with value: 0.8102046414743445 and parameters: {'n_estimators': 183, 'max_depth': 21, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.8155294831202671.
[I 2023-02-08 02:17:38,965] Trial 2 finished with value: 0.8005742233216588 and parameters: {'n_estimators': 183, 'max_depth': 15, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.8155294831202671.
[I 2023-02-08 02:17:49,610] Trial 3 finished with value: 0.809798002563733 and parameters: {'n_estimators': 145, 'max_depth': 21, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.8155294831202671.
[I 2023-02-08 02:17:55,266] Trial 4 finished with value: 0.8010538249779533 and parameters: {'n_

CPU times: user 7min 32s, sys: 12.6 s, total: 7min 45s
Wall time: 8min 46s


In [45]:
#Создаем модель с оптимальными параметрами
model_rfc04 = ensemble.RandomForestClassifier(**study02.best_params, random_state=random_state)

#Обучаем модель
model_rfc04.fit(X_train, y_train)

RandomForestClassifier(max_depth=16, min_samples_leaf=3, n_estimators=137,
                       random_state=42)

In [46]:
#Расчитываем метрики модели на тренировочной и тестовой выборках
print(f'Метрика f1 для RFC на выборке train: {metrics.f1_score(y_train, model_rfc04.predict(X_train))}')
print(f'Метрика f1 для RFC на выборке test: {metrics.f1_score(y_test, model_rfc04.predict(X_test))}')

Метрика f1 для RFC на выборке train: 0.9682151589242054
Метрика f1 для RFC на выборке test: 0.8014616321559075
